In [1]:
import cv2
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
import tensorflow as tf
from tqdm import tqdm

In [2]:
#download the data from here https://www.kaggle.com/xhlulu/140k-real-and-fake-faces
def build_model(pretrained):
    model = Sequential([
        pretrained,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(),
        metrics=['accuracy']
    )
    
    return model

# Generator

In [3]:
#this part is prolly the most trickiest one, make sure the base path is correct cause the code looks for the files
#respective to this path
base_path = './real-vs-fake/'

image_gen = ImageDataGenerator(rescale=1./255.)

train_flow = image_gen.flow_from_directory(
    base_path + 'train/',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

valid_flow = image_gen.flow_from_directory(
    base_path + 'valid/',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

test_flow = image_gen.flow_from_directory(
    base_path + 'test/',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

Found 100000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


# Train Model

In [4]:
#download the weight from here https://www.kaggle.com/xhlulu/densenet-keras
densenet = DenseNet121(
    weights='./DenseNet-BC-121-32-no-top.h5',
    include_top=False,
    input_shape=(224,224,3)
)
model = build_model(densenet)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 7, 7, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1025      
Total params: 7,038,529
Trainable params: 6,954,881
Non-trainable params: 83,648
_________________________________________________________________


In [5]:
train_steps = 100000//64
valid_steps = 20000//64

model.fit(
    train_flow,
    epochs=3,
    steps_per_epoch=train_steps,
    validation_data=valid_flow,
    validation_steps=valid_steps
)

Epoch 1/3
1562/1562 [==============================] - 59545s 38s/step - loss: 0.1209 - accuracy: 0.9505 - val_loss: 1.3828 - val_accuracy: 0.6277
Epoch 2/3
1562/1562 [==============================] - 29780s 19s/step - loss: 0.0479 - accuracy: 0.9823 - val_loss: 0.0908 - val_accuracy: 0.9698
Epoch 3/3
1562/1562 [==============================] - 28102s 18s/step - loss: 0.0355 - accuracy: 0.9870 - val_loss: 0.6224 - val_accuracy: 0.7776


# Evaluation

In [7]:
y_pred = model.predict(test_flow)
y_test = test_flow.classes

In [8]:
print("ROC AUC Score:", metrics.roc_auc_score(y_test, y_pred))
print("AP Score:", metrics.average_precision_score(y_test, y_pred))
print()
print(metrics.classification_report(y_test, y_pred > 0.5))

ROC AUC Score: 0.49576708
AP Score: 0.4930386632643057

              precision    recall  f1-score   support

           0       0.50      0.72      0.59     10000
           1       0.49      0.27      0.35     10000

    accuracy                           0.50     20000
   macro avg       0.49      0.50      0.47     20000
weighted avg       0.49      0.50      0.47     20000



In [10]:
model.evaluate(test_flow)

313/313 [==============================] - 2835s 9s/step - loss: 0.6369 - accuracy: 0.7770


[0.6368963122367859, 0.7769500017166138]

In [11]:
model.save("model.h5")

In [51]:
import keras
model = keras.models.load_model("model.h5")


frame = cv2.imread('trump.jpg')

frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

In [52]:
from PIL import Image
import numpy as np
from skimage import transform
def load(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')/255
   np_image = transform.resize(np_image, (224, 224, 3))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image

image = load('fake_1.jpg')
print(model.predict(image)[0][0])

1.1502162e-11
